# **Exploratory Data Analysis**: FlipTop

## Question
----

The central question of this project revolves around FlipTop battle rap careers and their length.

> How long does a career in the Philippines' premier rap battle league FlipTop last?

Who rises to the top? What determines who stays relevant over the years?

Here I'm thinking survival (time-to-event) analysis.

Another question I'm curious about is this:
> Who has battled who? And what matchups haven't been done yet?

I'm picturing a graph here. I want to understand how dense or sparse the network of rap battles are in FlipTop.

Ideas:
- Emcees are nodes
- "Battled" as edges
- Weight of the edge is how many times they've battled?

## Data Wrangling
----

Data wrangled using [YouTube API](https://developers.google.com/youtube/v3).

In [11]:
import sys
import json
import pandas as pd
import numpy as np
import isodate

Note: This data was scraped July 2, 2025.

In [2]:
# Load JSON data
with open("../data/videos.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Convert to DataFrame
df = pd.DataFrame(data)

# Preview
df.head()

,id,title,description,upload_date,view_count,duration,url,likeCount,commentCount,tags
0,NCElL6MbZ_g,FlipTop - Caspher vs CRhyme,FlipTop presents: Zoning 18 @ The TakeOver Lou...,2025-07-02T13:23:54Z,139884,PT32M32S,https://www.youtube.com/watch?v=NCElL6MbZ_g,3117,613,"[fliptop, flip top, flip top battles, fliptopb..."
1,JabvhPBmoVs,FlipTop CripLi - Isabuhay 2025 | Abangan si Cr...,"FlipTop presents: Gubat 15 @ Mariner's Court, ...",2025-06-25T13:49:25Z,27498,PT1M51S,https://www.youtube.com/watch?v=JabvhPBmoVs,640,31,"[fliptop, flip top, flip top battles, fliptopb..."
2,YiJI_ohq4Pc,FlipTop Ban - Isabuhay 2025 | Abangan si Ban n...,"FlipTop presents: Gubat 15 @ Mariner's Court, ...",2025-06-25T13:49:25Z,29353,PT1M24S,https://www.youtube.com/watch?v=YiJI_ohq4Pc,763,34,"[fliptop, flip top, flip top battles, fliptopb..."
3,yRE3PU0ekaA,FlipTop J-Blaque - Gods Pa Rin? | | Abangan si...,"FlipTop presents: Gubat 15 @ Mariner's Court, ...",2025-06-25T13:49:25Z,17415,PT1M56S,https://www.youtube.com/watch?v=yRE3PU0ekaA,360,8,"[fliptop, flip top, flip top battles, fliptopb..."
4,Ftk2DZ3hcxw,FlipTop Vitrum - Ang HipHop Pinalakas ng mga T...,"FlipTop presents: Gubat 15 @ Mariner's Court, ...",2025-06-25T13:48:58Z,28969,PT3M15S,https://www.youtube.com/watch?v=Ftk2DZ3hcxw,449,50,"[fliptop, flip top, flip top battles, fliptopb..."


In [3]:
sys.path.append("../scripts")

from emcee_scraper import scrape_names, write_names_to_csv
emcees = scrape_names()
write_names_to_csv(emcees, "../data/emcees.csv")

Scraped 20 names from page 1.
Scraped 20 names from page 2.
Scraped 20 names from page 3.
Scraped 20 names from page 4.
Scraped 20 names from page 5.
Scraped 20 names from page 6.
Scraped 20 names from page 7.
Scraped 20 names from page 8.
Scraped 7 names from page 9.
No emcees found on page 10. Stopping early.
Wrote 167 names to ../data/emcees.csv


## Data Cleaning
----

Tasks:

- convert `upload_date` to something more usable.
- convert `duration` to human readable (HH:MM:SS maybe?)
- Some shorts/round highlight moments were uploaded on the [videos playlist](https://www.youtube.com/@fliptopbattles/videos) of the channel so need to filter by duration, especially post-2020 (when YouTube Shorts became a thing).
- Only considering 1v1 battles that were judged (for now)
    - So remove:
        - Sound Check
        - Freestyle battles
        - Dos Por Dos (2v2)
        - Royal Rumble (1v1v1v1v1)
        - Those two 5v5 battles that happened only twice
        - Anygma Machine
        - Video Flyer
        - Announcement videos
        - Emcee interview videos

BIG TASKS

- Clean up `title` and create new column `battle_card` or something that contains only names e.g. Emcee1 vs Emcee2
- Webscrape [emcee page](https://www.fliptop.com.ph/emcees) to get a full and complete list of the emcees in FlipTop (according to their own website roster).
    - Good for later user of cross-referencing titles with emcee names.
- 

In [36]:
df.head()

,id,title,description,upload_date,view_count,duration,url,likeCount,commentCount,tags,duration_timedelta,duration_seconds
0,NCElL6MbZ_g,FlipTop - Caspher vs CRhyme,FlipTop presents: Zoning 18 @ The TakeOver Lou...,2025-07-02 13:23:54+00:00,139884,PT32M32S,https://www.youtube.com/watch?v=NCElL6MbZ_g,3117,613,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:32:32,1952.0
1,JabvhPBmoVs,FlipTop CripLi - Isabuhay 2025 | Abangan si Cr...,"FlipTop presents: Gubat 15 @ Mariner's Court, ...",2025-06-25 13:49:25+00:00,27498,PT1M51S,https://www.youtube.com/watch?v=JabvhPBmoVs,640,31,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:01:51,111.0
2,YiJI_ohq4Pc,FlipTop Ban - Isabuhay 2025 | Abangan si Ban n...,"FlipTop presents: Gubat 15 @ Mariner's Court, ...",2025-06-25 13:49:25+00:00,29353,PT1M24S,https://www.youtube.com/watch?v=YiJI_ohq4Pc,763,34,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:01:24,84.0
3,yRE3PU0ekaA,FlipTop J-Blaque - Gods Pa Rin? | | Abangan si...,"FlipTop presents: Gubat 15 @ Mariner's Court, ...",2025-06-25 13:49:25+00:00,17415,PT1M56S,https://www.youtube.com/watch?v=yRE3PU0ekaA,360,8,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:01:56,116.0
4,Ftk2DZ3hcxw,FlipTop Vitrum - Ang HipHop Pinalakas ng mga T...,"FlipTop presents: Gubat 15 @ Mariner's Court, ...",2025-06-25 13:48:58+00:00,28969,PT3M15S,https://www.youtube.com/watch?v=Ftk2DZ3hcxw,449,50,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:03:15,195.0


In [37]:
# looking at desc
samp = df.sample(n=5, replace=False)
samp

,id,title,description,upload_date,view_count,duration,url,likeCount,commentCount,tags,duration_timedelta,duration_seconds
1299,KHo2ALJ9xIk,FlipTop Isabubay 2015 + Grain Assault 6,FlipTop presents: Isabuhay 2015 and Grain Assa...,2015-01-15 14:22:07+00:00,255505,PT1M21S,https://www.youtube.com/watch?v=KHo2ALJ9xIk,619,381,"[isabuhay 3, fliptop, isabuhay 2015, grain ass...",0 days 00:01:21,81.0
899,UslUIIgeK9I,FlipTop - Towpher vs Emar Industriya,FlipTop Metro Manila Presents: Unibersikulo 6 ...,2018-12-05 12:16:52+00:00,1029651,PT22M27S,https://www.youtube.com/watch?v=UslUIIgeK9I,9816,2891,"[fliptop, fliptop new, fliptop latest, fliptop...",0 days 00:22:27,1347.0
1330,eG96pPRwD2Y,FlipTop - G-Clown vs Kregga,FlipTop Mindanao presents: Oro/Mata @ Beat Clu...,2014-08-11 13:38:19+00:00,416486,PT17M17S,https://www.youtube.com/watch?v=eG96pPRwD2Y,1037,328,"[g-clown, kregga, fliptop, fliptop new, flipto...",0 days 00:17:17,1037.0
668,1Vqr075jVlE,FlipTop - Romano UnggoYan,"FlipTop presents: UnggoYan @ Baraks, Mandaluyo...",2020-07-05 08:15:00+00:00,32818,PT1M50S,https://www.youtube.com/watch?v=1Vqr075jVlE,896,73,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:01:50,110.0
753,jtSy0niBR8o,FlipTop - LilJohn vs CripLi,https://fliptop.com.ph\nFlipTop Metro Cebu pre...,2019-11-29 12:33:05+00:00,2022527,PT26M31S,https://www.youtube.com/watch?v=jtSy0niBR8o,27248,4147,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:26:31,1591.0


In [38]:
samp["description"]

1299    FlipTop presents: Isabuhay 2015 and Grain Assa...
899     FlipTop Metro Manila Presents: Unibersikulo 6 ...
1330    FlipTop Mindanao presents: Oro/Mata @ Beat Clu...
668     FlipTop presents: UnggoYan @ Baraks, Mandaluyo...
753     https://fliptop.com.ph\nFlipTop Metro Cebu pre...
Name: description, dtype: object

In [39]:
# Write multi-line descriptions to file for analysis
title_desc = samp[["title", "description"]]
records = title_desc.to_dict(orient="records")

with open("../data/sample_descriptions.json", "w", encoding="utf-8") as f:
    json.dump(records, f, indent=2, ensure_ascii=False)

In [40]:
with open("../data/sample_descriptions.json", "r", encoding="utf-8") as f:
    descriptions = json.load(f)

# Pick the first one (index 0) or whichever index you want
sample = descriptions[1]

# Print nicely
print("TITLE:\n", sample["title"])
print("\nDESCRIPTION:\n", sample["description"])

TITLE:
 FlipTop - Towpher vs Emar Industriya

DESCRIPTION:
 FlipTop Metro Manila Presents: Unibersikulo 6 @ Quantum Makati, Bagtikan corner Kamagong, Makati City, Metro Manila, Philippines. October 20, 2018. Filipino Conference Battle.

-TOWPHER VS EMAR INDUSTRIYA-

Subscribe Here! http://bit.ly/fliptopsub
Check out our top videos! http://bit.ly/fliptopTopVideos

Website: https://fliptop.com.phFacebook: https://www.facebook.com/fliptop.battleleague
Twitter: https://twitter.com/FlipTop_Battles

#fliptopbattles

About fliptopbattles:
FlipTop Kru Corp. is a self-produced events and artist management company with its first product in the FlipTop Battle League. The FlipTop Battle League is the Philippines’ first premier – and the world’s most-viewed – rap battle league. It is popularly credited for the resurgence and widespread acceptance of hiphop culture in the Philippines since its inception in February 2010, and continues to champion all other hiphop elements in its variety of events an

Video descriptions often have good info on them for when and where the rap battle took place. Need to scrape these too!
- Let's put a pin on that right now. Do low-hanging fruit first before going for messy regex text extraction tasks.

In [41]:
df.head()

,id,title,description,upload_date,view_count,duration,url,likeCount,commentCount,tags,duration_timedelta,duration_seconds
0,NCElL6MbZ_g,FlipTop - Caspher vs CRhyme,FlipTop presents: Zoning 18 @ The TakeOver Lou...,2025-07-02 13:23:54+00:00,139884,PT32M32S,https://www.youtube.com/watch?v=NCElL6MbZ_g,3117,613,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:32:32,1952.0
1,JabvhPBmoVs,FlipTop CripLi - Isabuhay 2025 | Abangan si Cr...,"FlipTop presents: Gubat 15 @ Mariner's Court, ...",2025-06-25 13:49:25+00:00,27498,PT1M51S,https://www.youtube.com/watch?v=JabvhPBmoVs,640,31,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:01:51,111.0
2,YiJI_ohq4Pc,FlipTop Ban - Isabuhay 2025 | Abangan si Ban n...,"FlipTop presents: Gubat 15 @ Mariner's Court, ...",2025-06-25 13:49:25+00:00,29353,PT1M24S,https://www.youtube.com/watch?v=YiJI_ohq4Pc,763,34,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:01:24,84.0
3,yRE3PU0ekaA,FlipTop J-Blaque - Gods Pa Rin? | | Abangan si...,"FlipTop presents: Gubat 15 @ Mariner's Court, ...",2025-06-25 13:49:25+00:00,17415,PT1M56S,https://www.youtube.com/watch?v=yRE3PU0ekaA,360,8,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:01:56,116.0
4,Ftk2DZ3hcxw,FlipTop Vitrum - Ang HipHop Pinalakas ng mga T...,"FlipTop presents: Gubat 15 @ Mariner's Court, ...",2025-06-25 13:48:58+00:00,28969,PT3M15S,https://www.youtube.com/watch?v=Ftk2DZ3hcxw,449,50,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:03:15,195.0


In [42]:
# Convert to upload_date to datetime objects
df["upload_date"] = pd.to_datetime(df["upload_date"], utc=True)

In [43]:
df.head()

,id,title,description,upload_date,view_count,duration,url,likeCount,commentCount,tags,duration_timedelta,duration_seconds
0,NCElL6MbZ_g,FlipTop - Caspher vs CRhyme,FlipTop presents: Zoning 18 @ The TakeOver Lou...,2025-07-02 13:23:54+00:00,139884,PT32M32S,https://www.youtube.com/watch?v=NCElL6MbZ_g,3117,613,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:32:32,1952.0
1,JabvhPBmoVs,FlipTop CripLi - Isabuhay 2025 | Abangan si Cr...,"FlipTop presents: Gubat 15 @ Mariner's Court, ...",2025-06-25 13:49:25+00:00,27498,PT1M51S,https://www.youtube.com/watch?v=JabvhPBmoVs,640,31,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:01:51,111.0
2,YiJI_ohq4Pc,FlipTop Ban - Isabuhay 2025 | Abangan si Ban n...,"FlipTop presents: Gubat 15 @ Mariner's Court, ...",2025-06-25 13:49:25+00:00,29353,PT1M24S,https://www.youtube.com/watch?v=YiJI_ohq4Pc,763,34,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:01:24,84.0
3,yRE3PU0ekaA,FlipTop J-Blaque - Gods Pa Rin? | | Abangan si...,"FlipTop presents: Gubat 15 @ Mariner's Court, ...",2025-06-25 13:49:25+00:00,17415,PT1M56S,https://www.youtube.com/watch?v=yRE3PU0ekaA,360,8,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:01:56,116.0
4,Ftk2DZ3hcxw,FlipTop Vitrum - Ang HipHop Pinalakas ng mga T...,"FlipTop presents: Gubat 15 @ Mariner's Court, ...",2025-06-25 13:48:58+00:00,28969,PT3M15S,https://www.youtube.com/watch?v=Ftk2DZ3hcxw,449,50,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:03:15,195.0


In [44]:
# Create duration_timedelta col
df["duration_timedelta"] = df["duration"].apply(isodate.parse_duration)

# Create duration_seconds col (for ez calculations down the line)
df["duration_seconds"] = df["duration_timedelta"].dt.total_seconds()

In [45]:
df.head()

,id,title,description,upload_date,view_count,duration,url,likeCount,commentCount,tags,duration_timedelta,duration_seconds
0,NCElL6MbZ_g,FlipTop - Caspher vs CRhyme,FlipTop presents: Zoning 18 @ The TakeOver Lou...,2025-07-02 13:23:54+00:00,139884,PT32M32S,https://www.youtube.com/watch?v=NCElL6MbZ_g,3117,613,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:32:32,1952.0
1,JabvhPBmoVs,FlipTop CripLi - Isabuhay 2025 | Abangan si Cr...,"FlipTop presents: Gubat 15 @ Mariner's Court, ...",2025-06-25 13:49:25+00:00,27498,PT1M51S,https://www.youtube.com/watch?v=JabvhPBmoVs,640,31,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:01:51,111.0
2,YiJI_ohq4Pc,FlipTop Ban - Isabuhay 2025 | Abangan si Ban n...,"FlipTop presents: Gubat 15 @ Mariner's Court, ...",2025-06-25 13:49:25+00:00,29353,PT1M24S,https://www.youtube.com/watch?v=YiJI_ohq4Pc,763,34,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:01:24,84.0
3,yRE3PU0ekaA,FlipTop J-Blaque - Gods Pa Rin? | | Abangan si...,"FlipTop presents: Gubat 15 @ Mariner's Court, ...",2025-06-25 13:49:25+00:00,17415,PT1M56S,https://www.youtube.com/watch?v=yRE3PU0ekaA,360,8,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:01:56,116.0
4,Ftk2DZ3hcxw,FlipTop Vitrum - Ang HipHop Pinalakas ng mga T...,"FlipTop presents: Gubat 15 @ Mariner's Court, ...",2025-06-25 13:48:58+00:00,28969,PT3M15S,https://www.youtube.com/watch?v=Ftk2DZ3hcxw,449,50,"[fliptop, flip top, flip top battles, fliptopb...",0 days 00:03:15,195.0


Next task:
- Subset `df` to only include 1v1 battles.